In [ ]:
from foobah.gcode import GCODE
from foobah.utils import preview_svg
from foobah.methods import line_threshold
from foobah import constants
from foobah.constants import XMAX, XMIN, YMAX, YMIN

import PIL
import PIL.Image
import PIL.ImageFilter

import numpy as np

from random import uniform

from IPython.display import display

In [ ]:
basename = "foobar"
scale = 1
image_name = "images/square.png"
image_name = "images/david-bowie.jpg"
image = PIL.Image.open(image_name)

image = image.resize((int(image.width / scale), int(image.height / scale)), PIL.Image.ANTIALIAS)
# image = image.filter(PIL.ImageFilter.EDGE_ENHANCE_MORE)
# image = image.filter(PIL.ImageFilter.SMOOTH_MORE)
# image = image.filter(PIL.ImageFilter.CONTOUR)
# image = image.filter(PIL.ImageFilter.DETAIL)
# image = image.filter(PIL.ImageFilter.EDGE_ENHANCE)
# image = image.filter(PIL.ImageFilter.EDGE_ENHANCE_MORE)
# image = image.filter(PIL.ImageFilter.SMOOTH_MORE)

# image = image.rotate(-90, expand=True)

image = image.convert("RGB")

width = image.width
height = image.height

display(image)

In [ ]:
gcode = GCODE(basename, feedrate=3500)
gcode.move_to_starting_position()
gcode.pen_down()
gcode.draw_boundaries()
gcode.pen_up()

gcode.move_to_mid_point()
gcode.pen_up()

In [ ]:
def random_vec():
    return np.array((uniform(-1, 1), uniform(-1, 1)))
                    

def random_point():
    step_size = 20.0
    n_steps = 100
    
    position = np.array((uniform(0, image.width), uniform(0, image.height)))
    last_position = position + random_vec()
    
    gcode.pen_up()
    gcode.move_to(
        ((position[0]) / image.width) * (XMAX - XMIN) + XMIN,
        ((position[1]) / image.height) * (YMAX - YMIN) + YMIN,
    )
    gcode.pen_down()
    
    for _ in range(n_steps):
        if position[0] < 0 or position[0] >= image.width or position[1] < 0 or position[1] >= image.height:
            break
            
        r, g, b = image.getpixel((position[0], position[1]))
        intensity = ((255 - r) + (255 - g) + (255 - b)) / (255 * 3.0)

        step = (
            (np.array(random_vec()) * intensity) + 
            (
                (1.0 - intensity) * (position - last_position)
            )
        ) * step_size
        
        step = (
            (random_vec()) * (intensity)
        ) * step_size

#         print(f"{intensity:.4f} {step}")
        last_position = position.copy()
        position += step
        
        gcode.move_to(
            ((position[0]) / image.width) * (XMAX - XMIN) + XMIN,
            ((position[1]) / image.height) * (YMAX - YMIN) + YMIN,
        )

    gcode.pen_up()

In [ ]:
random_point()
for _ in range(1000):
    random_point()

In [ ]:
line_threshold(image, gcode, step_size=4)

In [ ]:
gcode.pen_up()
gcode.move_to_starting_position()
gcode.flush()

In [ ]:
preview_svg(basename)